# Featurize a dataset

Any machine learning model will expect tensorial representations of the chemical data. This notebooks provides a workflow to achieve such goal.

`kinoml.dataset.DatasetProvider` objects need to be available to deal with your collection of raw measurements for protein:ligand systems. These objects are, roughly, a list of `kinoml.core.BaseMeasurement`, each containing a set of `.values` and a some extra metadata, like the `system` objects to be featurized here.

In ligand-based models, protein information is only considered marginally, and most of the action happens at the ligand level. Usually starting with a string representation such as SMILES, or a database identifier such as a PubChem ID, these are promoted to (usually) RDKit objects and then transformed into a tensor of some form (e.g. fingerprints, molecular graph as an adjacency matrix, etc).

Available featurizers can be found under `kinoml.features`.

## How to use

Run `python run_notebook.py --help` for more information.

In [1]:
# If this is the template file (and not a copy) and you are introducing changes,
# update VERSION with the current date (YYYY.MM.DD)
VERSION = "2020.12.07" 

## ✏ Define hyper parameters

In [2]:
# TEMPLATE VALUES -- these are overriden (see below if executed) by papermill using a YAML or Python file as input
DATASET_CLS = "import.path.to.DatasetProvider"
DATASET_KWARGS = {"option": "value", "option2": "value2"}

PIPELINE = {
    "someuniquekey": [
        ("import.path.to.SomeFeaturizer", {"option": "value", "option2": "value2"}),
        ("import.path.to.SomeOtherFeaturizer", {"option": "value", "option2": "value2"}),
    ]
}
PIPELINE_AGG = "kinoml.features.core.Concatenated"
PIPELINE_AGG_KWARGS = {}

FEATURIZE_KWARGS = {"processes": 1}

GROUPS = [
    ("kinoml.datasets.groups.CallableGrouper", {"function": "lambda something: something.attribute"}),
    ("kinoml.datasets.groups.CallableGrouper", {"function": "lambda otherthing: otherthing.attribute2"})
]

TRAIN_TEST_VAL_KWARGS = {"idx_train": 0.8, "idx_test": 0.1, "idx_val": 0.1}

## IGNORE THIS ONE
HERE = _dh[-1]

In [3]:
# Parameters
DATASET_CLS = "kinoml.datasets.chembl.ChEMBLDatasetProvider"
DATASET_KWARGS = {}
PIPELINE = {
    "ligand": [
        ["kinoml.features.ligand.SmilesToLigandFeaturizer", {"style": "rdkit"}],
        [
            "kinoml.features.ligand.MorganFingerprintFeaturizer",
            {"nbits": 512, "radius": 2},
        ],
    ]
}
PIPELINE_AGG = "kinoml.features.core.Concatenated"
PIPELINE_AGG_KWARGS = {}
FEATURIZE_KWARGS = {"processes": 1}
GROUPS = [
    [
        "kinoml.datasets.groups.CallableGrouper",
        {"function": "lambda measurement: measurement.system.protein.name"},
    ],
    [
        "kinoml.datasets.groups.CallableGrouper",
        {"function": "lambda measurement: type(measurement).__name__"},
    ],
]
TRAIN_TEST_VAL_KWARGS = {"idx_train": 0.8, "idx_test": 0.1, "idx_val": 0.1}
HERE = "/Users/taliakimber/Documents/github/experiments-binding-affinity/features/ligand-only-morgan512"


⚠ From here on, you should _not_ need to modify anything else 🤞

---

Define key paths for data and outputs:

In [4]:
from pathlib import Path

HERE = Path(HERE)
for parent in HERE.parents:
    if next(parent.glob(".github/"), None):
        REPO = parent
        break

# Generate paths for this pipeline
featurizer_path = []
for name, branch in PIPELINE.items():
    featurizer_path.append(name)
    for clsname, kwargs in branch:
        clsname = clsname.rsplit(".", 1)[1]
        kwargs = [f"{k}={''.join(c for c in str(v) if c.isalnum())}" for k,v in kwargs.items()]
        featurizer_path.append("_".join([clsname] + kwargs))

OUT = HERE / "_output"  / "__".join(featurizer_path) / DATASET_CLS.rsplit('.', 1)[1]
OUT.mkdir(parents=True, exist_ok=True)

print(f"This notebook:           HERE = ~/{HERE.relative_to(Path.home())}")
print(f"This repo:               REPO = ~/{REPO.relative_to(Path.home())}")
print(f"Outputs in:               OUT = ~/{OUT.relative_to(Path.home())}")

This notebook:           HERE = ~/Documents/github/experiments-binding-affinity/features/ligand-only-morgan512
This repo:               REPO = ~/Documents/github/experiments-binding-affinity
Outputs in:               OUT = ~/Documents/github/experiments-binding-affinity/features/ligand-only-morgan512/_output/ligand__SmilesToLigandFeaturizer_style=rdkit__MorganFingerprintFeaturizer_nbits=512_radius=2/ChEMBLDatasetProvider


In [5]:
# Nasty trick: save all-caps local variables (CONSTANTS working as hyperparametrs) so far in a dict to save it later
_hparams = {key: value for key, value in locals().items() if key.upper() == key and not key.startswith(("_", "OE_"))}

## Setup is finished, start working

In [6]:
from warnings import warn
import os
import sys
from pathlib import Path
from datetime import datetime

import numpy as np

from kinoml.utils import seed_everything, import_object
seed_everything();
print("Run started at", datetime.now())

Run started at 2021-04-08 10:28:53.195569


## Load raw data

In [7]:
dataset = import_object(DATASET_CLS).from_source(**DATASET_KWARGS)
dataset

/opt/miniconda3/envs/experiments-binding-affinity/lib/python3.8/site-packages/kinoml/datasets/chembl.py:74: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df = df.assign(p_activities=-pd.np.log10(df["activities.standard_value"] * 1e-9))


  0%|          | 0/203380 [00:00<?, ?it/s]

<ChEMBLDatasetProvider with 203380 measurements (pIC50Measurement=170121, pKdMeasurement=17050, pKiMeasurement=16209), and 162584 systems (AminoAcidSequence=422, SmilesLigand=103097)>

In [8]:
df = dataset.to_dataframe()
df

,Systems,n_components,Measurement,MeasurementType
0,P00533 & Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F...,2,7.387216,pIC50Measurement
1,P35968 & Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F...,2,4.782516,pIC50Measurement
2,P00533 & Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)...,2,6.769551,pIC50Measurement
3,P06239 & Nc1ncnc2c1c(-c1cccc(Oc3ccccc3)c1)cn2C...,2,6.853872,pIC50Measurement
4,P06239 & Nc1ncnc2c1c(-c1cccc(Oc3ccccc3)c1)cn2C...,2,5.928118,pIC50Measurement
...,...,...,...,...
203375,P42345 & Nc1cc(C(F)F)c(-c2nc(N3CCOCC3)cc(N3CCO...,2,7.376751,pKiMeasurement
203376,P42345 & Nc1cc(C(F)(F)F)c(-c2cc(N3C4CCC3COC4)n...,2,7.522879,pKiMeasurement
203377,P42345 & Nc1cc(C(F)F)c(-c2cc(N3C4CCC3COC4)nc(N...,2,7.920819,pKiMeasurement
203378,P42345 & Nc1cc(C(F)(F)F)c(-c2nc(N3C4CCC3COC4)c...,2,6.361511,pKiMeasurement


## Featurize

In [9]:
# build pipeline
from kinoml.features.core import Pipeline

featurizers = []
for key, featurizer_instructions in PIPELINE.items():
    featurizers.append(Pipeline([import_object(import_str)(**kwargs) for import_str, kwargs in featurizer_instructions]))
featurizer = import_object(PIPELINE_AGG)(featurizers, **PIPELINE_AGG_KWARGS)

In [10]:
# prefeaturize everything
dataset.featurize(featurizer, **FEATURIZE_KWARGS);

  0%|          | 0/162584 [00:00<?, ?it/s]

## Filter

Remove systems that couldn't be featurized. Successful featurizations are stored in `measurement.system.featurizations['last']` so we test for that key existence.

In [11]:
from kinoml.datasets.groups import CallableGrouper, RandomGrouper
grouper = CallableGrouper(lambda measurement: 'invalid' if 'last' not in measurement.system.featurizations else 'valid')
grouper.assign(dataset, overwrite=True, progress=False)
groups = dataset.split_by_groups()
if "invalid" in groups:
    _invalid = groups.pop("invalid")
    warn(f"{len(_invalid)} entries could not be featurized!. Possible errors:")
    warn(f"{_invalid[0].system.featurizations}")

## Groups

Cumulatively apply groups.

In [12]:
groups[("valid",)] = groups.pop("valid")
for grouper_str, grouper_kwargs in GROUPS:
    grouper_cls = import_object(grouper_str)
    ## We need this because lambda functions are not JSON-serializable
    if issubclass(grouper_cls, CallableGrouper):
        for k, v in list(grouper_kwargs.items()):
            if k == "function" and isinstance(v, str):
                grouper_kwargs[k] = eval(v)  # sorry :)
    ## End of lambda hack
    grouper = grouper_cls(**grouper_kwargs)        
    for group_key in list(groups.keys()):
        grouper.assign(groups[group_key], overwrite=True, progress=False)
        for subkey, subgroup in groups.pop(group_key).split_by_groups().items():
            groups[group_key + (subkey,)] = subgroup
print("10 groups to show keys:", *list(groups.keys())[:10], sep="\n")

10 groups to show keys:
('valid', 'P00533', 'pIC50Measurement')
('valid', 'P00533', 'pKdMeasurement')
('valid', 'P00533', 'pKiMeasurement')
('valid', 'P35968', 'pIC50Measurement')
('valid', 'P35968', 'pKdMeasurement')
('valid', 'P35968', 'pKiMeasurement')
('valid', 'P06239', 'pIC50Measurement')
('valid', 'P06239', 'pKdMeasurement')
('valid', 'P06239', 'pKiMeasurement')
('valid', 'Q02763', 'pIC50Measurement')


## Write tensors to disk

Output files are written to `_output/<PIPELINE>/<DATASET>/<GROUP>.npz` files.

Each `npz` will contain two `np.ndarray` objects: `X` (featurized systems) and `y` (associated measurements), plus the train/test/validation indices.

In [13]:
random_grouper = RandomGrouper(TRAIN_TEST_VAL_KWARGS)

for group, ds in sorted(groups.items(), key=lambda kv: len(kv[1]), reverse=True):
    indices = random_grouper.indices(ds)
    X = np.asarray(ds.featurized_systems())
    y = ds.measurements_as_array()
    np.savez(OUT / f"{'__'.join([g for g in group if g != 'valid'])}.npz", X=X, y=y.astype("float32"), **indices)

In [14]:
print("Run finished at", datetime.now())

Run finished at 2021-04-08 10:31:28.920420


# Reproducibility logs

In [15]:
from kinoml.utils import watermark
w = watermark()

Watermark
---------
Last updated: 2021-04-08T10:31:28.978176+02:00

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

Compiler    : Clang 11.0.1 
OS          : Darwin
Release     : 20.2.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Hostname: talias-macbook-pro.home

Git hash: 2fd8a9e16d0bd0ce05e34e1358a8e9706c8648ed

numpy: 1.20.2
sys  : 3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:12:38) 
[Clang 11.0.1 ]

Watermark: 2.2.0


conda
-----


sys.version: 3.7.7 (default, Mar 23 2020, 17:31:31) 
...
sys.prefix: /opt/miniconda3
sys.executable: /opt/miniconda3/bin/python
conda location: /opt/miniconda3/lib/python3.7/site-packages/conda
conda-build: None
conda-env: /opt/miniconda3/bin/conda-env
user site dirs: 

CIO_TEST: <not set>
CONDA_DEFAULT_ENV: experiments-binding-affinity
CONDA_EXE: /opt/miniconda3/bin/conda
CONDA_MKL_INTERFACE_LAYER_BACKUP: 
CONDA_PREFIX: /opt/miniconda3/envs/experiments-binding-affinity
CONDA_PREFIX_1: /opt/miniconda3
CONDA_PROMPT_MODIFIER: (experiments-binding-affinity) 
CONDA_PYTHON_EXE: /opt/miniconda3/bin/python
CONDA_ROOT: /opt/miniconda3
CONDA_SHLVL: 2
CURL_CA_BUNDLE: <not set>
PATH: /opt/miniconda3/envs/experiments-binding-affinity/bin:/opt/miniconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/Library/Apple/usr/bin
PYTHONHASHSEED: 1234
REQUESTS_CA_BUNDLE: <not set>
SSL_CERT_FILE: <not set>




# packages in environment at /opt/miniconda3/envs/experiments-binding-affinity:
#
# Name                    Version                   Build  Channel
_py-xgboost-mutex         2.0                       cpu_0    conda-forge
absl-py                   0.12.0             pyhd8ed1ab_0    conda-forge
aiohttp                   3.7.4            py38h96a0964_0    conda-forge
alabaster                 0.7.12                   pypi_0    pypi
amberlite                 16.0                     pypi_0    pypi
ambertools                20.15                    pypi_0    pypi
ansiwrap                  0.8.4                      py_0    conda-forge
anyio                     2.2.0            py38h50d1736_0    conda-forge
appdirs                   1.4.4              pyh9f0ad1d_0    conda-forge
appnope                   0.1.2            py38h50d1736_1    conda-forge
argon2-cffi               20.1.0           py38h5406a74_2    conda-forge
arpack                    3.7.0                hefb7bc6_2    conda-fo

In [16]:
%%capture cap --no-stderr
w = watermark()

In [17]:
import json

with open(OUT/ "watermark.txt", "w") as f:
    f.write(cap.stdout)

with open(OUT / "hparams.json", "w") as f:
    json.dump(_hparams, f, default=str, indent=2)